# Hackerrank

Julia code for solving various hackerrank problems. For reproduction purpose mostly.

## Algorithms

### Search

#### Ice cream parlor

Solved 

In [ ]:
n_trips = parse(Int, readline())

function read_trip()
    budget = parse(Int, readline())
    n = parse(Int, readline())
    prices = [parse(Int,i) for i in split(readline())]
    a,b = solve(budget, n, prices)
    println(string(min(a,b))*" "*string(max(a,b)))
end

function solve(budget, n, prices)
    for i in 1:n
        for j in (i+1):n
            if prices[i]+prices[j] == budget return (i,j) end
        end
    end
end

for i in 1:n_trips read_trip() end

#### KnightL on a Chessboard

Solved

In [31]:
add(x,y)=x+y
sub(x,y)=x-y

function minimum_length(a::Int8,b::Int8,n)
    accessible, counter, finished = Set([(0,0)]), 1, false 
    last_new = Set([(0,0)])
    while !finished
        new_accessible::Set{Tuple{Int8, Int8}} = Set()
        for old_pos in last_new
            for (f1,f2) in [(a,b),(b,a)]
                for oper1 in [add,sub] for oper2 in [add,sub]
                        newx::Int8 = oper1(old_pos[1],f1)
                        newy::Int8 = oper2(old_pos[2],f2)
                        if 0 ≤ newx < n && 0 ≤ newy < n 
                            new_accessible = ∪(new_accessible, Set([(newx,newy)]))
                        end end
                end
            end
        end
        if (n-1,n-1) in new_accessible 
            return counter
        end
        last_new = new_accessible
        new_accessible = ∪(new_accessible,accessible)
        if accessible == new_accessible finished = true end
        accessible = new_accessible
        counter +=1
    end
    return -1
end

function main(N)
result_matrix = Array{Int8}(N-1,N-1)
for i::Int8 in 1:(N-1)
        if (N-1) % i == 0 
            result_matrix[i,i] = (N-1) / i 
        else result_matrix[i,i] = -1 end
    for j::Int8 in (i+1):(N-1)
        newres = minimum_length(i,j,N)
        result_matrix[i,j] = newres
        result_matrix[j,i] = newres
        end
    end
for i in 1:(N-1) 
    for j in 1:(N-1)
        tmp = result_matrix[i,j]
        print("$tmp ")
    end
    print("\n")
end
end

@time main(5)


4 4 2 8 
4 2 4 4 
2 4 -1 -1 
8 4 -1 1 
  0.141289 seconds (28.40 k allocations: 1.472 MiB)


### Graph theory

#### Roads and Libraries

Efficient DFS implementation to get the number of subgraph in the graph

In [1]:
type GraphNode
    connexions::Set{Int}
    marked::Bool
end
GraphNode() = GraphNode(Set(), false)

type NOGraph
    nodes::Array{GraphNode}
    unmarked_min::Int
end
NOGraph(size::Int) = NOGraph([GraphNode() for i in 1:size], 1)

function add_connexion(graph::NOGraph, from::Int, to::Int)
    push!(graph.nodes[from].connexions, to)
    push!(graph.nodes[to].connexions, from)
end

function get_unmarked(graph::NOGraph)::Int
    for nd_id in graph.unmarked_min:length(graph.nodes)
        if !graph.nodes[nd_id].marked
            graph.unmarked_min = nd_id+1
            return nd_id 
        end
    end
    return -1
end

function split_connected(graph::NOGraph)
    n_cycles, next_node = 0, get_unmarked(graph)
    while next_node ≠ -1
        DFS(graph, next_node)
        n_cycles +=1
        next_node = get_unmarked(graph)
    end
    return n_cycles
end     
    
function DFS(graph::NOGraph, nd_id::Int)
    stack = [nd_id]
    while length(stack) > 0
        new_node = pop!(stack)
        graph.nodes[new_node].marked = true
        for i in graph.nodes[new_node].connexions
            if !graph.nodes[i].marked
                push!(stack, i)
                graph.nodes[i].marked = true
            end
        end
    end
end

######################## PROBLEM #########################

q = parse(Int, readline())
function do_one()
    n_cities, n_roads, c_lib, c_road = map(parse,split(readline()))
    mygraph = NOGraph(n_cities)
    for i in 1:n_roads
        from, to = map(x -> parse(Int,x),split(readline()))
        add_connexion(mygraph, from, to)
    end
    if c_lib < c_road 
        return c_lib*n_cities
    else
    n_subgraph = split_connected(mygraph)
    return c_lib*n_subgraph + c_road*(n_cities-n_subgraph)
    end
end

for i in 1:q
    println(do_one())
end

Set([7])

In [42]:
M, N, R = map(x->parse(Int, x), split(readline()))
arr = Array{Int}(M,N)
for i in 1:M
    tmp = map(x->parse(Int, x), split(readline()))
    for j in 1:N
        arr[i,j] = tmp[j]
    end
end

function unroll(x,m,n)
    n_rolls = div(min(m,n),2)
    result = []
    for i in 1:n_rolls
        tmp = []
        for d1 in i:(m-i)
            push!(tmp, x[d1,i])
        end
        for d2 in i:(n-i+1)
            push!(tmp, x[m-i+1,d2])
        end
        for d1 in i:(m-i)
            push!(tmp, x[m-d1,n-i+1])
        end
        for d2 in i:(n-i-1)
            push!(tmp, x[i,n-d2])
        end
        push!(result, tmp)
    end
    return result
end

function roll(rolls,m,n)
    x = [reverse(e) for e in rolls]
    n_rolls = div(min(m,n),2)
    result = Array{Int}(m,n)
    for i in 1:n_rolls
        for d1 in i:(m-i)
            result[d1,i] = pop!(x[i])
        end
        for d2 in i:(n-i+1)
            result[m-i+1,d2] = pop!(x[i])
        end
        for d1 in i:(m-i)
            result[m-d1,n-i+1] = pop!(x[i])
        end
        for d2 in i:(n-i-1)
            result[i,n-d2] = pop!(x[i])
        end
    end
    return result
end

function rotate(mat, R, m, n)
    rolls = unroll(mat,m,n)
    function rotate_one(arr, R)
        md = R % length(arr)
        return vcat(arr[end-md+1:end], arr[1:end-md])
    end
    rotated_rolls = [rotate_one(e, R) for e in rolls]
    return roll(rotated_rolls,m,n)
end

function smart_print(mat,m,n)
    for i in 1:m
        for j in 1:n
            print("$(mat[i,j]) ")
        end
        print("\n")
    end
end

my_result = rotate(arr, R, M, N)

smart_print(my_result, M, N)

[1 2 3 4; 5 6 7 8; 9 10 11 12; 13 14 15 16]


#### Matrix Layer Rotation

In [66]:
function unroll(x,m,n)
    n_rolls = div(min(m,n),2)
    result = []
    for i in 1:n_rolls
        tmp = []
        for d1 in i:(m-i)
            push!(tmp, x[d1,i])
        end
        for d2 in i:(n-i+1)
            push!(tmp, x[m-i+1,d2])
        end
        for d1 in i:(m-i)
            push!(tmp, x[n-d1,m-i+1])
        end
        for d2 in i:(n-i-1)
            push!(tmp, x[i,n-d2])
        end
        push!(result, tmp)
    end
    return result
end

function roll(rolls,m,n)
    x = [reverse(e) for e in rolls]
    n_rolls = div(min(m,n),2)
    result = Array{Int}(m,n)
    for i in 1:n_rolls
        for d1 in i:(m-i)
            result[d1,i] = pop!(x[i])
        end
        for d2 in i:(n-i+1)
            result[m-i+1,d2] = pop!(x[i])
        end
        for d1 in i:(m-i)
            result[n-d1,m-i+1] = pop!(x[i])
        end
        for d2 in i:(n-i-1)
            result[i,n-d2] = pop!(x[i])
        end
    end
    return result
end

function rotate(mat, R, m, n)
    rolls = unroll(mat,m,n)
    function rotate_one(arr, R)
        md = R % length(arr)
        return vcat([arr[end-md+1:end], arr[1:end-md]])
    end
    rotated_rolls [rotate_one(e, R) for e in rolls]
    return roll(rotated_rolls,m,n)
end

Any[1, 5, 9, 13, 14, 15, 16, 12, 8, 4, 3, 2]
[1 2 3 4; 5 6 7 8; 9 10 11 12; 13 14 15 16]


In [79]:
function get_powers()
    x, result = parse(BigInt, "2"), ["1"]
    for i in 1:801
        push!(result, string(x))
        x*=2
    end
    return result
end

powersof2 = get_powers()

function do_one(f)
    str = readline(f)
    cnt = 0
    for pwr in powersof2
        cnt += count(x->true, eachmatch(Regex(pwr), str, true))
    end
    println(cnt)
end

function main()
    open("twotwo.txt") do f
    Q = parse(Int, readline(f))
    for i in 1:Q do_one(f) end
    end
end

@time main()

83396
83499
83430
83251
83299
83297
83399
83236
83362
83233
83323
83356
83313
83126
83533
83011
83228
83215
83372
83278
83335
83536
83294
83357
83162
83283
83344
83346
83368
83272
83221
83378
83189
83422
83216
83259
83140
83508
83318
83392
83354
83340
83376
83304
83176
83354
83154
83309
83267
83353
83156
83367
83400
83395
83131
83237
83582
83564
83101
83515
83362
83327
83354
83433
83294
83371
83483
83326
83320
83322
83344
83395
83352
83165
83247
83289
83471
83374
83321
83121
83083
83232
83387
83416
83189
83390
83481
83370
83169
83100
83340
83286
83355
83312
83235
83297
83367
83256
83387
83533
 11.952098 seconds (42.32 M allocations: 2.155 GiB, 2.52% gc time)
